In [1]:
import requests
import pandas as pd
import json
import warnings

Henter datakatalog-definisjon

In [2]:
r = requests.get( 'https://www.vegvesen.no/nvdb/api/v2/vegobjekttyper', headers={'Accept' : 'application/json'},
                params={'inkluder' : 'egenskapstyper'})
dakat = r.json()
dakat2 = {}
for dd in dakat: 
    dakat2[dd['navn']] = dd 

Leser regneark

In [41]:
df = pd.read_excel( 'NVDB mapping av datatyper til ITS direktivet-V3.xlsx')
df.fillna(' ', inplace=True)
df.dtypes


NVDB datatyper ( relatert til ITS direktivet)    object
Mobilitetstilbud                                 object
 Veg og vegregulering                            object
Trafikkinformasjon                               object
Description                                      object
Kommentar                                        object
dtype: object

## husk å justere antall kolonner, kolonnenavn 

In [7]:
excelnavn = df.columns[0]
mobil = df.columns[1]
vegregulering = df.columns[2]
trafikkinfo  = df.columns[3]
desc = df.columns[4]

# Funksjonsdefinisjon

Vi leser innholdet av los1.md, los2.md og los3.md, tar vare på all tekst oppforbi tabellen og lager ny tabell ut fra regnearket vi nettopp leste innf. Bokstaven "x" i riktig kolonne angir hvilke kategorier en objekttype inngår i. 

Vi kobler NVDB-navnet på objekttypen mot datakatalog-definisjonen, og lenker til denne. Dette steget tjener også som en kvalitetssjekk på at NVDB-navnet er riktig. 

In [45]:
# Funksjonsdefinisjon
def lesfil( filename, skilletegn='Feature type |  Description'): 
    """
    Leser fil inntil vi treffer på forekomst av teksten skilletegn
    """
    
    allelinjer = ''
    
    with open( filename) as f:
        eilinje  = f.readline()
        while eilinje and skilletegn not in eilinje: 
            allelinjer += eilinje
            eilinje = f.readline()
            
    return allelinjer

def prettyprint( df, df_name_column, filename, dakat): 
    """
    Matcher regneark-navn i kolonne mot datakatalog-definisjon og skriver ut en 
    pen markdown-tabell
    """
    with open( filename, 'w', encoding='utf-8') as f:
        f.write( '| Feature type |  Description | Selected properties |Description  |\n')
        f.write( '|---|---|---|---|\n')
        for index, row in df.iterrows(): 
            print( row[df_name_column])
            if row[df_name_column] in dakat:
                mindef = dakat[row[df_name_column]]
                f.write( '| [' + str( mindef['id']  ) + ' ' + mindef['navn'] + 
                    '](https://datakatalogen.vegdata.no/' + str(mindef['id']) + ') | | |  |\n' )
                for eg in mindef['egenskapstyper']:
                    f.write( '| | | ' + str(eg['id']) + ' ' + eg['navn'] + ' | |\n ')
                
            else: 
                f.write( '| | | | |\n')
                f.write( '|**' + row[df_name_column] + '**| **FANT IKKE I DATAKATALOGEN** | | |\n' )
                f.write( '| | | | |\n')
                warnings.warn( 'Fant ikke i datakatalogen' +  row[df_name_column] )
          
        
def simpleprint( df, df_name_column, filename, dakat): 
    """
    Som prettyprint, men uten egenskapnavn 
    """
    
    # leser inn øverste del av fila (inntil tabellen begynner)
    start = lesfil( filename)
    
    with open( filename, 'w', encoding='utf-8') as f:
        f.write(start )
        f.write( '| Feature type |  Description | \n')
        f.write( '|---|---|\n')
        for index, row in df.iterrows(): 
            print( row[df_name_column])
            if row[df_name_column].strip() in dakat:
                mindef = dakat[row[df_name_column].strip()]
                f.write( '| [' + str( mindef['id']  ) + ' ' + mindef['navn'] + 
                    '](https://datakatalogen.vegdata.no/' + str(mindef['id']) + ') | '+
                    row['Description']  + ' |\n' )

            else: 
                f.write( '| | |\n')
                f.write( '|**' + row[df_name_column] + '**| **FANT IKKE I DATAKATALOGEN** |\n' )
                f.write( '| | |\n')
                warnings.warn( row[df_name_column] + ' Fant ikke i datakatalogen' )


# Skriver nye tabeller 

Her skriver vi nye tabeller for de tre LOS kategoriene.  

In [38]:
simpleprint( df[ df[mobil] == 'x' ], excelnavn, '../los1.md', dakat2)

Døgnhvileplass
Ferjeleie
Fortau
Gangadkomst
Parkeringsområde
Rasteplass
Sykkelparkering
Sykkeltilbud, riksvegrute
Toalettanlegg
Turistveg
Undergang


In [46]:
simpleprint( df[ df[trafikkinfo] == 'x' ], excelnavn, '../los2.md', dakat2 )

ATK-punkt
Bomstasjon 
Fartsgrense
Fartsgrense, variabel
Fartstavle
Forbikjøringsstrekning
Forkjørsveg
Fortau
Gangadkomst
Gangfelt
Gågate
Høydebegrensning
Jernbanekryssing
Kolonnestrekning
Kontroll-/veieplass
Omkjøringsrute
Parkeringsområde
Rasteplass
Snuplass
Streknings-ATK
Trafikklomme
Trafikkreguleringer
Variabelt skilt
Vegoppmerking, langsgående
Vegoppmerking, tverrgående


In [40]:
simpleprint( df[ df[vegregulering] == 'x'  ], excelnavn, '../los3.md', dakat2 )

ATK-punkt
Bru
Bruksklasse, modulvogntog
Bruksklasse, normaltransport
Bruksklasse, spesialtransport
Bruksklasse, tømmertransport
Fartsgrense
Fartsgrense, variabel
Fartstavle
Faunapassasje
Forbikjøringsstrekning
Forkjørsveg
Fortau
Gangadkomst
Gangfelt
Gate
Gågate
Høydebegrensning
Jernbanekryssing
Kolonnestrekning
Kontroll-/veieplass
Kurvatur, horisontalelement
Kurvatur, stigning
Kurvatur, vertikalelement
Motorveg
Omkjøringsrute
Rasteplass
Snuplass
Streknings-ATK
Sykkelrute, hovednett by/tettsted
Sykkelrute, tur/fritid
TEN-T veg
Trafikkberedskapsklasse
Trafikklomme
Trafikkreguleringer
Tunnel
Tunnelløp
Turistveg
Undergang
Variabelt skilt
Vegbredde
Vegbredde, beregnet
Vegkryss
Vegoppmerking, langsgående
Vegoppmerking, tverrgående
